# Generators

# 生成器

Here we'll take a deeper dive into Python generators, including *generator expressions* and *generator functions*.

本章我们深入讨论Python的生成器，包括*生成器表达式*和*生成器函数*

## Generator Expressions

## 生成器表达式

The difference between list comprehensions and generator expressions is sometimes confusing; here we'll quickly outline the differences between them:

列表解析和生成器表达式之间的区别很容易令人混乱；下面我们快速地说明一下它们之间的区别：

### List comprehensions use square brackets, while generator expressions use parentheses

### 列表解析使用中括号，而生成器表达式使用小括号

This is a representative list comprehension:

下面是一个很有代表性的列表解析：

In [1]:
[n ** 2 for n in range(12)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]

While this is a representative generator expression:

下面这个确是一个生成器表达式：

In [2]:
(n ** 2 for n in range(12))

<generator object <genexpr> at 0x7f0c405b75e8>

Notice that printing the generator expression does not print the contents; one way to print the contents of a generator expression is to pass it to the ``list`` constructor:

你会注意到直接打印生成器表达式并不会输出生成器的内容；可以使用`list`将生成器转换为一个列表然后输出：

In [3]:
G = (n ** 2 for n in range(12))
list(G)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]

### A list is a collection of values, while a generator is a recipe for producing values

### 列表是一个集合，而生成器只是产生集合值的配方

When you create a list, you are actually building a collection of values, and there is some memory cost associated with that.
When you create a generator, you are not building a collection of values, but a recipe for producing those values.
Both expose the same iterator interface, as we can see here:

当你创建一个列表，你真实地创建了一个集合，当然这个集合存储在内存当中需要一定的空间。当你创建了一个生成器，你并没有创建一个集合，你仅仅是指定了产生集合值的方法。两者都实现了迭代器接口，由下面两个例子可以看到：

In [4]:
L = [n ** 2 for n in range(12)]
for val in L:
    print(val, end=' ')

0 1 4 9 16 25 36 49 64 81 100 121 

In [5]:
G = (n ** 2 for n in range(12))
for val in G:
    print(val, end=' ')

0 1 4 9 16 25 36 49 64 81 100 121 

The difference is that a generator expression does not actually compute the values until they are needed.
This not only leads to memory efficiency, but to computational efficiency as well!
This also means that while the size of a list is limited by available memory, the size of a generator expression is unlimited!

区别在于生成器仅在你用到值的时候才会按照配方计算一个值返回给你。这样的好处不仅仅是节省内存，还能节省计算资源。这还意味着，列表的大小受限于可用内存的大小，而生成器的大小是无限的。

An example of an infinite generator expression can be created using the ``count`` iterator defined in ``itertools``:

我们可以使用`itertools`里面的`count`函数来构造一个无限的生成器表达式：

In [6]:
from itertools import count
count()

count(0)

In [7]:
for i in count():
    print(i, end=' ')
    if i >= 10: break

0 1 2 3 4 5 6 7 8 9 10 

The ``count`` iterator will go on happily counting forever until you tell it to stop; this makes it convenient to create generators that will also go on forever:

`count`函数会永远的迭代下去除非你停止了它的运行；这也可以用来创建永远运行的生成器：

In [8]:
factors = [2, 3, 5, 7]
G = (i for i in count() if all(i % n > 0 for n in factors))
for val in G:
    print(val, end=' ')
    if val > 40: break

1 11 13 17 19 23 29 31 37 41 

You might see what we're getting at here: if we were to expand the list of factors appropriately, what we would have the beginnings of is a prime number generator, using the Sieve of Eratosthenes algorithm. We'll explore this more momentarily.

上面的例子你应该已经看出来了：如果我们使用Sieve of Eratosthenes算法，将factors列表进行合适的扩展的话，那么我们将会得到一个质数的生成器。

### A list can be iterated multiple times; a generator expression is single-use

### 列表可以被迭代多次；生成器只能是一次使用

This is one of those potential gotchas of generator expressions.
With a list, we can straightforwardly do this:

这是生成器的一个著名的坑。使用列表时，我们可以如下做：

In [9]:
L = [n ** 2 for n in range(12)]
for val in L:
    print(val, end=' ')
print()

for val in L:
    print(val, end=' ')

0 1 4 9 16 25 36 49 64 81 100 121 
0 1 4 9 16 25 36 49 64 81 100 121 

A generator expression, on the other hand, is used-up after one iteration:

生成器表达式则不一样，只能迭代一次：

In [10]:
G = (n ** 2 for n in range(12))
list(G)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]

In [11]:
list(G)

[]

This can be very useful because it means iteration can be stopped and started:

这是非常有用的特性，因为这意味着迭代能停止和开始：

In [13]:
G = (n**2 for n in range(12))
for n in G:
    print(n, end=' ')
    if n > 30: break  # 生成器停止运行

print("\ndoing something in between")

for n in G:  # 生成器继续运行
    print(n, end=' ')

0 1 4 9 16 25 36 
doing something in between
49 64 81 100 121 

One place I've found this useful is when working with collections of data files on disk; it means that you can quite easily analyze them in batches, letting the generator keep track of which ones you have yet to see.

作者发现这个特性在使用磁盘上存储的数据文件时特别有用；它意味着你可以很容易的按批次来分析数据，让生成器记录下目前的处理进度。

## Generator Functions: Using ``yield``

## 生成器函数：使用 `yield`

We saw in the previous section that list comprehensions are best used to create relatively simple lists, while using a normal ``for`` loop can be better in more complicated situations.
The same is true of generator expressions: we can make more complicated generators using *generator functions*, which make use of the ``yield`` statement.

从上面的讨论中，我们可以知道列表解析适用于创建相对简单的列表，如果列表的生成规则比较复杂，还是使用普通`for`循环更加合适。对于生成器表达式来说也一样：我们可以使用*生成器函数*创建更加复杂的生成器，这里需要用到`yield`关键字。

Here we have two ways of constructing the same list:

我们有两种方式来构建同一个列表：

In [14]:
L1 = [n ** 2 for n in range(12)]

L2 = []
for n in range(12):
    L2.append(n ** 2)

print(L1)
print(L2)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]


Similarly, here we have two ways of constructing equivalent generators:

类似的，我们也有两种方法来构建相同的生成器：

In [15]:
G1 = (n ** 2 for n in range(12))

def gen():
    for n in range(12):
        yield n ** 2

G2 = gen()
print(*G1)
print(*G2)

0 1 4 9 16 25 36 49 64 81 100 121
0 1 4 9 16 25 36 49 64 81 100 121


A generator function is a function that, rather than using ``return`` to return a value once, uses ``yield`` to yield a (potentially infinite) sequence of values.
Just as in generator expressions, the state of the generator is preserved between partial iterations, but if we want a fresh copy of the generator we can simply call the function again.

生成器函数与普通函数的区别在于，生成器函数不是使用`return`来一次性返回值，而是使用`yield`来产生一系列（可能无穷多个）值。就像生成器表达式一样，生成器的状态会被生成器自己保留并记录，如果你需要一个新的生成器，你可以再次调用函数。

## Example: Prime Number Generator

## 例子：质数生成器

Here I'll show my favorite example of a generator function: a function to generate an unbounded series of prime numbers.
A classic algorithm for this is the *Sieve of Eratosthenes*, which works something like this:

下面作者将介绍他最喜欢的生成器函数的例子：一个可以产生无穷多个质数序列的函数。计算质数又一个经典算法*Sieve of Eratosthenes*，它的工作原理如下：

In [16]:
# 产生可能的质数序列
L = [n for n in range(2, 40)]
print(L)

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]


In [17]:
# 剔除所有被第一个元素整除的数
L = [n for n in L if n == L[0] or n % L[0] > 0]
print(L)

[2, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39]


In [18]:
# 剔除所有被第二个元素整除的数
L = [n for n in L if n == L[1] or n % L[1] > 0]
print(L)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 25, 29, 31, 35, 37]


In [19]:
# 剔除所有被第三个元素整除的数
L = [n for n in L if n == L[2] or n % L[2] > 0]
print(L)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37]


If we repeat this procedure enough times on a large enough list, we can generate as many primes as we wish.

如果我们在一个很大的列表上重复这个过程足够多次，我们可以生成我们需要的质数。

Let's encapsulate this logic in a generator function:

我们将这个逻辑封装到一个生成器函数中：

In [20]:
def gen_primes(N):
    """Generate primes up to N"""
    primes = set()  # 使用primes集合存储找到的质数
    for n in range(2, N):
        if all(n % p > 0 for p in primes):  # primes中的元素都不能整除n -> n是质数
            primes.add(n)  # 将n加入primes集合
            yield n  # 产生序列

print(*gen_primes(100))

2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97


That's all there is to it!
While this is certainly not the most computationally efficient implementation of the Sieve of Eratosthenes, it illustrates how convenient the generator function syntax can be for building more complicated sequences.

虽然这可能不是最优化的Sieve of Eratosthenes算法实现，但是它表明声称其函数语法是多么简便，而且可以用来构建很复杂的序列。